<a href="https://colab.research.google.com/github/abdelaziz2003vvb/Enhanced-RAG-System-with-DeepEval-Evaluation/blob/main/course_rag_finale_one.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# =============================================================================
# SECTION 1: INSTALLATION & IMPORTS
# ==============================================================

In [1]:
print("📦 Installing required packages...")

!pip install -q langchain langchain-community langchain-cohere langchain-text-splitters
!pip install -q cohere faiss-cpu pypdf python-docx
!pip install -q sentence-transformers tiktoken

print("✅ All packages installed successfully!\n")

# Imports
import os
import time
from typing import List, Dict, Tuple
import numpy as np
from google.colab import files, drive

# LangChain
from langchain_community.document_loaders import (
    PyPDFLoader,
    Docx2txtLoader,
    TextLoader,
    DirectoryLoader
)
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain_cohere import CohereEmbeddings, ChatCohere

# Cohere
import cohere

print("✅ All imports successful!\n")

📦 Installing required packages...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.0/319.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.2/328.2 kB 7.4 MB/s e

✅ All imports successful!



# =============================================================================
# SECTION 2: CONFIGURATION
# =================================================================

In [2]:
class Config:
    """Configuration for RAG System"""

    # API Keys
    COHERE_API_KEY = "ViqgIQ4U0D64Md9QFeMaInVvD5m0cSX2bhvA8otF"

    # Models
    COHERE_MODEL = "command-r-plus-08-2024"
    COHERE_EMBEDDING_MODEL = "embed-english-v3.0"
    COHERE_RERANK_MODEL = "rerank-english-v3.0"

    # Chunking Parameters
    CHUNK_SIZE = 1000
    CHUNK_OVERLAP = 200

    # Retrieval Parameters
    TOP_K = 10  # Retrieve more for reranking
    TOP_K_RERANK = 5  # Final number after reranking

    # FAISS Parameters
    FAISS_INDEX_TYPE = "IndexFlatL2"  # Exact search (or "IndexIVFFlat" for faster ANN)

    # Paths
    VECTOR_STORE_PATH = "./faiss_index"
    UPLOAD_FOLDER = "./uploaded_documents"

config = Config()

# Set environment variable
os.environ["COHERE_API_KEY"] = config.COHERE_API_KEY

print("⚙️ Configuration loaded successfully!")
print(f"   - Model: {config.COHERE_MODEL}")
print(f"   - Embedding Model: {config.COHERE_EMBEDDING_MODEL}")
print(f"   - Chunk Size: {config.CHUNK_SIZE}")
print(f"   - Top K Retrieval: {config.TOP_K}")
print()

⚙️ Configuration loaded successfully!
   - Model: command-r-plus-08-2024
   - Embedding Model: embed-english-v3.0
   - Chunk Size: 1000
   - Top K Retrieval: 10



# =============================================================================
# SECTION 3: DOCUMENT LOADING
# =================================================================


In [3]:
class DocumentLoader:
    """Load documents from various formats"""

    def __init__(self):
        self.supported_formats = ['.pdf', '.docx', '.txt', '.md']

    def upload_files(self) -> List[str]:
        """Upload files from local computer"""
        print("📤 Please upload your documents...")
        uploaded = files.upload()

        # Create upload directory
        os.makedirs(config.UPLOAD_FOLDER, exist_ok=True)

        file_paths = []
        for filename in uploaded.keys():
            file_path = os.path.join(config.UPLOAD_FOLDER, filename)
            with open(file_path, 'wb') as f:
                f.write(uploaded[filename])
            file_paths.append(file_path)
            print(f"   ✅ Uploaded: {filename}")

        return file_paths

    def load_from_drive(self, drive_path: str) -> List[str]:
        """Load files from Google Drive"""
        drive.mount('/content/drive')

        file_paths = []
        for root, dirs, files in os.walk(drive_path):
            for file in files:
                if any(file.endswith(ext) for ext in self.supported_formats):
                    file_paths.append(os.path.join(root, file))

        print(f"📁 Found {len(file_paths)} documents in Drive")
        return file_paths

    def load_documents(self, file_paths: List[str]) -> List[Document]:
        """Load documents from file paths"""
        all_documents = []

        for file_path in file_paths:
            try:
                # Select appropriate loader based on file extension
                if file_path.endswith('.pdf'):
                    loader = PyPDFLoader(file_path)
                elif file_path.endswith('.docx'):
                    loader = Docx2txtLoader(file_path)
                elif file_path.endswith(('.txt', '.md')):
                    loader = TextLoader(file_path)
                else:
                    print(f"   ⚠️ Unsupported format: {file_path}")
                    continue

                docs = loader.load()

                # Add metadata
                for doc in docs:
                    doc.metadata['source'] = os.path.basename(file_path)
                    doc.metadata['file_path'] = file_path

                all_documents.extend(docs)
                print(f"   ✅ Loaded: {os.path.basename(file_path)} ({len(docs)} pages)")

            except Exception as e:
                print(f"   ❌ Error loading {file_path}: {str(e)}")

        print(f"\n📚 Total documents loaded: {len(all_documents)}\n")
        return all_documents

# =============================================================================
# SECTION 4: TEXT CHUNKING
# =========================================================

In [4]:
class TextChunker:
    """Split documents into chunks"""

    def __init__(self, chunk_size: int = config.CHUNK_SIZE,
                 chunk_overlap: int = config.CHUNK_OVERLAP):
        self.splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            length_function=len,
            separators=["\n\n", "\n", ". ", " ", ""]
        )

    def clean_text(self, text: str) -> str:
        """Clean and normalize text"""
        # Remove excessive whitespace
        text = " ".join(text.split())
        return text

    def chunk_documents(self, documents: List[Document]) -> List[Document]:
        """Split documents into chunks"""
        print("✂️ Chunking documents...")

        # Clean documents
        for doc in documents:
            doc.page_content = self.clean_text(doc.page_content)

        # Split into chunks
        chunks = self.splitter.split_documents(documents)

        # Add chunk metadata
        for i, chunk in enumerate(chunks):
            chunk.metadata['chunk_id'] = i
            chunk.metadata['chunk_size'] = len(chunk.page_content)

        print(f"   ✅ Created {len(chunks)} chunks")
        print(f"   📊 Average chunk size: {np.mean([c.metadata['chunk_size'] for c in chunks]):.0f} characters\n")

        return chunks


# =============================================================================
# SECTION 5: EMBEDDINGS & VECTOR STORE (FAISS)
# ==============================================================

In [5]:
class FAISSVectorStore:
    """Manage FAISS vector store with Cohere embeddings"""

    def __init__(self):
        print("🔧 Initializing Cohere Embeddings...")
        self.embeddings = CohereEmbeddings(
            cohere_api_key=config.COHERE_API_KEY,
            model=config.COHERE_EMBEDDING_MODEL
        )
        self.vectorstore = None
        print("   ✅ Embeddings initialized\n")

    def create_vectorstore(self, chunks: List[Document]) -> FAISS:
        """Create FAISS vector store from chunks"""
        print("🔢 Creating embeddings and building FAISS index...")
        print(f"   Processing {len(chunks)} chunks...")

        start_time = time.time()

        # Create FAISS vector store
        self.vectorstore = FAISS.from_documents(
            documents=chunks,
            embedding=self.embeddings
        )

        elapsed_time = time.time() - start_time

        print(f"   ✅ FAISS index created in {elapsed_time:.2f} seconds")
        print(f"   📊 Index size: {self.vectorstore.index.ntotal} vectors")
        print(f"   📏 Vector dimension: {self.vectorstore.index.d}\n")

        return self.vectorstore

    def save_vectorstore(self, path: str = config.VECTOR_STORE_PATH):
        """Save FAISS index to disk"""
        if self.vectorstore is None:
            raise ValueError("No vector store to save")

        print(f"💾 Saving FAISS index to {path}...")
        self.vectorstore.save_local(path)
        print("   ✅ Vector store saved\n")

    def load_vectorstore(self, path: str = config.VECTOR_STORE_PATH):
        """Load FAISS index from disk"""
        print(f"📂 Loading FAISS index from {path}...")

        self.vectorstore = FAISS.load_local(
            path,
            self.embeddings,
            allow_dangerous_deserialization=True
        )

        print(f"   ✅ Loaded {self.vectorstore.index.ntotal} vectors\n")
        return self.vectorstore

    def similarity_search(self, query: str, k: int = config.TOP_K) -> List[Document]:
        """Search for similar documents using FAISS ANN"""
        if self.vectorstore is None:
            raise ValueError("Vector store not initialized")

        return self.vectorstore.similarity_search(query, k=k)

    def similarity_search_with_score(self, query: str, k: int = config.TOP_K) -> List[Tuple[Document, float]]:
        """Search with similarity scores"""
        if self.vectorstore is None:
            raise ValueError("Vector store not initialized")

        return self.vectorstore.similarity_search_with_score(query, k=k)

# =============================================================================
# SECTION 6: RERANKING (COHERE)
# =========================================================

In [6]:
class CohereReranker:
    """Rerank retrieved documents using Cohere"""

    def __init__(self):
        self.client = cohere.Client(config.COHERE_API_KEY)
        self.model = config.COHERE_RERANK_MODEL

    def rerank(self, query: str, documents: List[Document],
               top_k: int = config.TOP_K_RERANK) -> List[Tuple[Document, float]]:
        """Rerank documents using Cohere reranking API"""

        if not documents:
            return []

        print(f"🎯 Reranking {len(documents)} documents...")

        # Prepare documents for reranking
        doc_texts = [doc.page_content for doc in documents]

        try:
            # Call Cohere rerank API
            response = self.client.rerank(
                model=self.model,
                query=query,
                documents=doc_texts,
                top_n=min(top_k, len(documents)),
                return_documents=False
            )

            # Create reranked results
            reranked = []
            for result in response.results:
                doc_index = result.index
                relevance_score = result.relevance_score

                doc = documents[doc_index]
                doc.metadata['rerank_score'] = relevance_score
                doc.metadata['rerank_position'] = len(reranked) + 1

                reranked.append((doc, relevance_score))

            print(f"   ✅ Reranked to top {len(reranked)} documents\n")
            return reranked

        except Exception as e:
            print(f"   ⚠️ Reranking error: {e}")
            print("   📝 Returning original order\n")
            return [(doc, 0.5) for doc in documents[:top_k]]

# =============================================================================
# SECTION 7: LLM GENERATION (COHERE)
# ==========================================================

In [7]:
class LLMGenerator:
    """Generate answers using Cohere LLM"""

    def __init__(self):
        self.llm = ChatCohere(
            cohere_api_key=config.COHERE_API_KEY,
            model=config.COHERE_MODEL,
            temperature=0.3,
            max_tokens=1000
        )

    def generate_answer(self, query: str, context_docs: List[Document]) -> Dict:
        """Generate answer from query and context"""

        # Prepare context
        context = "\n\n".join([
            f"Document {i+1} (Source: {doc.metadata.get('source', 'Unknown')}):\n{doc.page_content}"
            for i, doc in enumerate(context_docs)
        ])

        # Create prompt
        prompt = f"""Use the following context from documents to answer the question at the end.
If you don't know the answer based on the context, just say that you don't know, don't try to make up an answer.
Always cite the source documents in your answer.

Context:
{context}

Question: {query}

Detailed Answer:"""

        print("🤖 Generating answer with Cohere LLM...")
        start_time = time.time()

        # Generate response
        response = self.llm.invoke(prompt)

        generation_time = time.time() - start_time

        print(f"   ✅ Answer generated in {generation_time:.2f} seconds\n")

        return {
            "answer": response.content,
            "sources": context_docs,
            "generation_time": generation_time
        }


# =============================================================================
# SECTION 8: COMPLETE RAG PIPELINE
# =========================================================


In [8]:
class RAGPipeline:
    """Complete RAG pipeline orchestration"""

    def __init__(self):
        self.vector_store = FAISSVectorStore()
        self.reranker = CohereReranker()
        self.generator = LLMGenerator()
        self.is_initialized = False

    def ingest_documents(self, file_paths: List[str]):
        """Complete ingestion pipeline"""
        print("="*70)
        print("STEP 1: DOCUMENT LOADING")
        print("="*70)

        loader = DocumentLoader()
        documents = loader.load_documents(file_paths)

        print("="*70)
        print("STEP 2: TEXT CHUNKING")
        print("="*70)

        chunker = TextChunker()
        chunks = chunker.chunk_documents(documents)

        # FIX: Store chunk count in app_state
        app_state.total_chunks = len(chunks)

        print("="*70)
        print("STEP 3: CREATING EMBEDDINGS & FAISS INDEX")
        print("="*70)

        self.vector_store.create_vectorstore(chunks)

        print("="*70)
        print("STEP 4: SAVING VECTOR STORE")
        print("="*70)

        self.vector_store.save_vectorstore()

        self.is_initialized = True

        print("="*70)
        print("✅ INGESTION COMPLETE!")
        print("="*70)
        print()

    def load_existing_index(self):
        """Load existing FAISS index"""
        self.vector_store.load_vectorstore()
        self.is_initialized = True

    def query(self, question: str, use_reranking: bool = True) -> Dict:
        """Complete RAG query pipeline"""

        if not self.is_initialized:
            raise ValueError("Pipeline not initialized. Run ingest_documents() first.")

        print("="*70)
        print(f"QUERY: {question}")
        print("="*70)

        # STEP 1: Retrieve from FAISS
        print("\n📊 STEP 1: RETRIEVAL (FAISS ANN)")
        print("-"*70)

        retrieval_start = time.time()
        retrieved_docs = self.vector_store.similarity_search(question, k=config.TOP_K)
        retrieval_time = time.time() - retrieval_start

        print(f"   Retrieved {len(retrieved_docs)} documents in {retrieval_time:.3f}s")

        # STEP 2: Rerank
        if use_reranking:
            print("\n🎯 STEP 2: RERANKING (COHERE)")
            print("-"*70)

            rerank_start = time.time()
            reranked_docs = self.reranker.rerank(question, retrieved_docs, config.TOP_K_RERANK)
            rerank_time = time.time() - rerank_start

            final_docs = [doc for doc, score in reranked_docs]
            print(f"   Reranked to {len(final_docs)} documents in {rerank_time:.3f}s")
        else:
            final_docs = retrieved_docs[:config.TOP_K_RERANK]
            rerank_time = 0

        # STEP 3: Generate answer
        print("\n🤖 STEP 3: GENERATION (COHERE LLM)")
        print("-"*70)

        result = self.generator.generate_answer(question, final_docs)

        # Add timing information
        result['retrieval_time'] = retrieval_time
        result['rerank_time'] = rerank_time
        result['total_time'] = retrieval_time + rerank_time + result['generation_time']

        print("="*70)
        print("✅ QUERY COMPLETE")
        print("="*70)

        return result

    def display_result(self, result: Dict):
        """Display query result in a nice format"""

        print("\n" + "="*70)
        print("📝 ANSWER")
        print("="*70)
        print(result['answer'])

        print("\n" + "="*70)
        print("📚 SOURCES")
        print("="*70)

        for i, doc in enumerate(result['sources'], 1):
            print(f"\n{i}. Source: {doc.metadata.get('source', 'Unknown')}")
            print(f"   Rerank Score: {doc.metadata.get('rerank_score', 'N/A')}")
            print(f"   Content Preview: {doc.page_content[:200]}...")

        print("\n" + "="*70)
        print("⏱️ PERFORMANCE METRICS")
        print("="*70)
        print(f"   Retrieval Time: {result['retrieval_time']:.3f}s")
        print(f"   Rerank Time: {result['rerank_time']:.3f}s")
        print(f"   Generation Time: {result['generation_time']:.3f}s")
        print(f"   Total Time: {result['total_time']:.3f}s")
        print("="*70)

#=============================================================================
#SECTION 9:DEEPEVAL EVALUATION + HUMAN-IN-THE-LOOP EXTENSION
#=================================================================



# CELL 1: Install Deep eval

In [9]:
!pip install datasets --upgrade --quiet
!pip install deepeval --upgrade --quiet --break-system-packages
from datasets import Dataset
import json
from datetime import datetime
import pandas as pd



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.3/512.3 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 794.9/794.9 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 2.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not current


# CELL 2: Enhanced AppState with Feedback Storage

In [10]:
class EnhancedAppState:
    """Extended AppState with evaluation and feedback capabilities"""

    def __init__(self):
        # Existing attributes
        self.rag_pipeline = None
        self.chat_history = []
        self.query_stats = []
        self.is_initialized = False
        self.documents_loaded = 0
        self.file_paths = []
        self.total_chunks = 0

        # NEW: Feedback storage
        self.feedback_data = []
        self.evaluation_results = []

        # NEW: Ground truth Q&A pairs
        self.ground_truth_qa = []

    def add_feedback(self, query_id, feedback_type, rating=None, comment=None, corrected_answer=None):
        """Store user feedback"""
        feedback = {
            'timestamp': datetime.now().isoformat(),
            'query_id': query_id,
            'feedback_type': feedback_type,  # 'thumbs_up', 'thumbs_down', 'correction'
            'rating': rating,  # 1-5 stars
            'comment': comment,
            'corrected_answer': corrected_answer,
            'question': self.chat_history[query_id]['question'] if query_id < len(self.chat_history) else None,
            'original_answer': self.chat_history[query_id]['answer'] if query_id < len(self.chat_history) else None
        }
        self.feedback_data.append(feedback)
        print(f"✅ Feedback recorded: {feedback_type}")
        return feedback

    def export_feedback(self, filename="feedback_data.json"):
        """Export all feedback data"""
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(self.feedback_data, f, indent=2, ensure_ascii=False)
        return filename

    def get_feedback_summary(self):
        """Get feedback statistics"""
        if not self.feedback_data:
            return "No feedback yet"

        thumbs_up = sum(1 for f in self.feedback_data if f['feedback_type'] == 'thumbs_up')
        thumbs_down = sum(1 for f in self.feedback_data if f['feedback_type'] == 'thumbs_down')
        corrections = sum(1 for f in self.feedback_data if f['feedback_type'] == 'correction')

        ratings = [f['rating'] for f in self.feedback_data if f['rating'] is not None]
        avg_rating = sum(ratings) / len(ratings) if ratings else 0

        return {
            'total_feedback': len(self.feedback_data),
            'thumbs_up': thumbs_up,
            'thumbs_down': thumbs_down,
            'corrections': corrections,
            'average_rating': avg_rating,
            'satisfaction_rate': (thumbs_up / (thumbs_up + thumbs_down) * 100) if (thumbs_up + thumbs_down) > 0 else 0
        }

# Replace the existing app_state
app_state = EnhancedAppState()


# CELL 3 :DEEPEVAL IMPLEMENTATION - Complete Replacement for RAGAS


In [11]:
print("="*70)
print("🚀 COMPLETE DEEPEVAL SETUP")
print("="*70)

# Step 1: Imports
print("\n📦 Step 1: Importing libraries...")
from deepeval.metrics import (
    AnswerRelevancyMetric,
    FaithfulnessMetric,
    ContextualPrecisionMetric,
    ContextualRecallMetric
)
from deepeval.test_case import LLMTestCase
from deepeval.models.base_model import DeepEvalBaseLLM
from langchain_cohere import ChatCohere
from datetime import datetime
import pandas as pd
import json

print("✅ Imports complete")

# Step 2: Create Cohere Model for DeepEval
print("\n🔧 Step 2: Creating Cohere model...")

class CohereModel(DeepEvalBaseLLM):
    """Custom Cohere model for DeepEval"""

    def __init__(self, api_key, model="command-r-plus-08-2024"):
        self.model_name = model
        self.llm = ChatCohere(
            cohere_api_key=api_key,
            model=model,
            temperature=0
        )

    def load_model(self):
        return self.llm

    def generate(self, prompt: str) -> str:
        try:
            response = self.llm.invoke(prompt)
            return response.content
        except Exception as e:
            print(f"   ❌ Generate error: {e}")
            raise

    async def a_generate(self, prompt: str) -> str:
        return self.generate(prompt)

    def get_model_name(self):
        return self.model_name

# Create the Cohere model
cohere_model = CohereModel(api_key=config.COHERE_API_KEY)
print("✅ Cohere model created")

# Test the model
print("   Testing model...")
try:
    test_response = cohere_model.generate("Say 'OK' in one word")
    print(f"   ✅ Model test: {test_response}")
except Exception as e:
    print(f"   ⚠️ Model test failed: {e}")

# Step 3: Create DeepEval Evaluator
print("\n🔧 Step 3: Creating DeepEval Evaluator...")

class DeepEvalEvaluator:
    """Fixed DeepEval evaluator with proper context handling"""

    def __init__(self, model):
        self.model = model

        # Initialize metrics
        print("   Initializing Faithfulness...")
        self.faithfulness = FaithfulnessMetric(
            threshold=0.7,
            model=self.model,
            include_reason=True
        )

        print("   Initializing Answer Relevancy...")
        self.answer_relevancy = AnswerRelevancyMetric(
            threshold=0.7,
            model=self.model,
            include_reason=True
        )

        print("   Initializing Contextual Precision...")
        self.contextual_precision = ContextualPrecisionMetric(
            threshold=0.7,
            model=self.model,
            include_reason=True
        )

        print("   Initializing Contextual Recall...")
        self.contextual_recall = ContextualRecallMetric(
            threshold=0.7,
            model=self.model,
            include_reason=True
        )

        print("   ✅ All metrics initialized")

    def evaluate_single_query(self, question, answer, contexts, ground_truth=None):
        """
        Evaluate a single query-answer pair using DeepEval
        """
        print(f"\n{'='*70}")
        print(f"🔬 DEEPEVAL EVALUATION")
        print(f"{'='*70}")
        print(f"Question: {question[:80]}...")

        # Validate inputs
        if not question or not answer:
            print("❌ Question or answer is empty!")
            return None

        # Ensure contexts is a list of strings
        if not isinstance(contexts, list):
            contexts = [str(contexts)]

        contexts = [str(c).strip() for c in contexts if c and str(c).strip()]

        if not contexts:
            print("❌ No valid contexts provided!")
            return None

        print(f"📄 Contexts: {len(contexts)} documents")
        print(f"📏 Total context length: {sum(len(c) for c in contexts)} characters")

        has_ground_truth = ground_truth and str(ground_truth).strip()
        if has_ground_truth:
            print(f"✅ Ground truth provided")
        else:
            print(f"ℹ️ No ground truth (using 2 basic metrics)")

        try:
            # Create test case
            test_case = LLMTestCase(
                input=question,
                actual_output=answer,
                retrieval_context=contexts,
                expected_output=ground_truth if has_ground_truth else None
            )

            print(f"\n⏳ Running DeepEval metrics...")

            # Run metrics
            scores = {}

            # Faithfulness
            print(f"   📊 Evaluating Faithfulness...")
            try:
                self.faithfulness.measure(test_case)
                scores['faithfulness'] = self.faithfulness.score
                print(f"      ✅ Score: {self.faithfulness.score:.4f}")
            except Exception as e:
                print(f"      ❌ Failed: {str(e)[:100]}")

            # Answer Relevancy
            print(f"   📊 Evaluating Answer Relevancy...")
            try:
                self.answer_relevancy.measure(test_case)
                scores['answer_relevancy'] = self.answer_relevancy.score
                print(f"      ✅ Score: {self.answer_relevancy.score:.4f}")
            except Exception as e:
                print(f"      ❌ Failed: {str(e)[:100]}")

            # Ground truth metrics if available
            if has_ground_truth:
                print(f"   📊 Evaluating Contextual Precision...")
                try:
                    self.contextual_precision.measure(test_case)
                    scores['contextual_precision'] = self.contextual_precision.score
                    print(f"      ✅ Score: {self.contextual_precision.score:.4f}")
                except Exception as e:
                    print(f"      ❌ Failed: {str(e)[:100]}")

                print(f"   📊 Evaluating Contextual Recall...")
                try:
                    self.contextual_recall.measure(test_case)
                    scores['contextual_recall'] = self.contextual_recall.score
                    print(f"      ✅ Score: {self.contextual_recall.score:.4f}")
                except Exception as e:
                    print(f"      ❌ Failed: {str(e)[:100]}")

            if not scores:
                print(f"\n❌ No metrics succeeded!")
                return None

            print(f"\n✅ Evaluation complete!")
            print(f"\n📊 FINAL SCORES:")
            for metric, score in scores.items():
                emoji = "✅" if score >= 0.7 else "⚠️" if score >= 0.5 else "❌"
                print(f"   {emoji} {metric}: {score:.4f}")

            print(f"{'='*70}\n")

            return scores

        except Exception as e:
            print(f"\n❌ EVALUATION ERROR:")
            print(f"   {str(e)}")
            import traceback
            print(f"\n📋 Traceback: {traceback.format_exc()[:500]}...")
            print(f"{'='*70}\n")
            return None

    def evaluate_batch(self, qa_pairs):
        """Evaluate multiple Q&A pairs"""
        print(f"\n{'='*70}")
        print(f"🔬 BATCH DEEPEVAL EVALUATION")
        print(f"{'='*70}")
        print(f"Total pairs: {len(qa_pairs)}")

        # Validate inputs
        valid_pairs = []
        for i, qa in enumerate(qa_pairs):
            if not qa.get('question') or not qa.get('answer'):
                print(f"⚠️ Skipping pair {i+1}: missing question or answer")
                continue

            contexts = qa.get('contexts', [])
            if not isinstance(contexts, list):
                contexts = [str(contexts)]

            contexts = [str(c).strip() for c in contexts if c and str(c).strip()]

            if not contexts:
                print(f"⚠️ Skipping pair {i+1}: no valid contexts")
                continue

            valid_pairs.append({
                'question': qa['question'],
                'answer': qa['answer'],
                'contexts': contexts,
                'ground_truth': qa.get('ground_truth', '')
            })

        if not valid_pairs:
            print("❌ No valid pairs to evaluate!")
            return None

        print(f"✅ Valid pairs: {len(valid_pairs)}")

        # Evaluate each pair
        all_scores = {
            'faithfulness': [],
            'answer_relevancy': []
        }

        has_ground_truth = all(
            pair.get('ground_truth') and str(pair['ground_truth']).strip()
            for pair in valid_pairs
        )

        if has_ground_truth:
            all_scores['contextual_precision'] = []
            all_scores['contextual_recall'] = []

        for i, pair in enumerate(valid_pairs, 1):
            print(f"\n[{i}/{len(valid_pairs)}] {pair['question'][:60]}...")

            scores = self.evaluate_single_query(
                question=pair['question'],
                answer=pair['answer'],
                contexts=pair['contexts'],
                ground_truth=pair.get('ground_truth')
            )

            if scores:
                for metric, score in scores.items():
                    if metric in all_scores:
                        all_scores[metric].append(score)

        # Calculate averages
        avg_scores = {}
        for metric, scores_list in all_scores.items():
            if scores_list:
                avg_scores[metric] = sum(scores_list) / len(scores_list)

        print(f"\n{'='*70}")
        print(f"📊 AVERAGE SCORES:")
        for metric, score in avg_scores.items():
            emoji = "✅" if score >= 0.7 else "⚠️" if score >= 0.5 else "❌"
            print(f"   {emoji} {metric}: {score:.4f}")
        print(f"{'='*70}\n")

        return avg_scores

    def generate_report(self, scores, output_file=None):
        """Generate detailed evaluation report"""

        if not scores:
            return "❌ No scores available"

        report = f"""
╔═══════════════════════════════════════════════════════════════╗
║                   DEEPEVAL EVALUATION REPORT                  ║
║                  {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}                          ║
╚═══════════════════════════════════════════════════════════════╝

METRIC SCORES
{'='*60}

"""
        for metric, score in scores.items():
            status = "✅" if score >= 0.7 else "⚠️" if score >= 0.5 else "❌"
            metric_name = metric.replace('_', ' ').title()
            report += f"{status} {metric_name:<30} {score:.4f}\n"

        report += f"\n{'='*60}\n"

        # Overall assessment
        avg_score = sum(scores.values()) / len(scores)
        report += f"OVERALL SCORE: {avg_score:.4f}\n\n"

        if avg_score >= 0.8:
            report += "🏆 EXCELLENT - System performing very well!\n"
        elif avg_score >= 0.7:
            report += "✅ GOOD - System performing adequately\n"
        elif avg_score >= 0.5:
            report += "⚠️ FAIR - System needs improvement\n"
        else:
            report += "❌ POOR - Significant improvements needed\n"

        report += f"{'='*60}\n"

        if output_file:
            try:
                with open(output_file, 'w', encoding='utf-8') as f:
                    f.write(report)
            except:
                pass

        return report


# Create the evaluator
evaluator = DeepEvalEvaluator(model=cohere_model)

print("\n" + "="*70)
print("✅ COMPLETE SETUP SUCCESSFUL!")
print("="*70)
print("\n📊 Available Metrics:")
print("   ✅ Faithfulness (always)")
print("   ✅ Answer Relevancy (always)")
print("   ✅ Contextual Precision (with ground truth)")
print("   ✅ Contextual Recall (with ground truth)")
print("\n💡 Ready to use!")
print("="*70)

# Quick test
print("\n🧪 Running quick test...")
if app_state.is_initialized:
    try:
        result = app_state.rag_pipeline.query("MFCC signifie quoi?", use_reranking=True)
        contexts = [doc.page_content for doc in result['sources']]

        scores = evaluator.evaluate_single_query(
            question="MFCC signifie quoi?",
            answer=result['answer'],
            contexts=contexts,
            ground_truth="MFCC est l'acronyme de Mel Frequency Cepstral Coefficients"
        )

        if scores:
            print("\n🎉 TEST SUCCESSFUL! Evaluator is working!")
        else:
            print("\n⚠️ Test completed but no scores returned")
    except Exception as e:
        print(f"\n⚠️ Test error: {str(e)[:200]}")
else:
    print("⚠️ System not initialized - skipping test")
    print("   Initialize system first, then test evaluation")

print("\n" + "="*70)

🚀 COMPLETE DEEPEVAL SETUP

📦 Step 1: Importing libraries...
✅ Imports complete

🔧 Step 2: Creating Cohere model...
✅ Cohere model created
   Testing model...
   ✅ Model test: OK.

🔧 Step 3: Creating DeepEval Evaluator...
   Initializing Faithfulness...
   Initializing Answer Relevancy...
   Initializing Contextual Precision...
   Initializing Contextual Recall...
   ✅ All metrics initialized

✅ COMPLETE SETUP SUCCESSFUL!

📊 Available Metrics:
   ✅ Faithfulness (always)
   ✅ Answer Relevancy (always)
   ✅ Contextual Precision (with ground truth)
   ✅ Contextual Recall (with ground truth)

💡 Ready to use!

🧪 Running quick test...
⚠️ System not initialized - skipping test
   Initialize system first, then test evaluation




# CELL 4: Enhanced Query Function with Evaluation


In [12]:
def query_with_evaluation(question, use_reranking, top_k, evaluate_response=False, ground_truth=None):
    """Enhanced query function with DeepEval evaluation"""

    if not app_state.is_initialized:
        return "❌ Please initialize the system first!", None, "System not initialized", None, None

    if not question or not question.strip():
        return "❌ Please enter a question!", None, "Empty question", None, None

    try:
        print(f"\n{'='*70}")
        print(f"🔍 PROCESSING QUERY")
        print(f"{'='*70}")
        print(f"Question: {question}")
        print(f"Reranking: {use_reranking}")
        print(f"Evaluate: {evaluate_response}")

        # Query the RAG pipeline
        print(f"\n⏳ Querying RAG pipeline...")
        result = app_state.rag_pipeline.query(question, use_reranking=use_reranking)

        # Extract contexts as clean strings
        contexts = []
        for doc in result['sources']:
            context_text = doc.page_content.strip()
            if context_text:
                contexts.append(context_text)

        print(f"✅ Retrieved {len(contexts)} context documents")

        # Initialize evaluation variables
        evaluation_scores = None
        evaluation_report = "ℹ️ Evaluation not requested"

        # Run DeepEval evaluation if requested
        if evaluate_response:
            print(f"\n🔬 Running DeepEval evaluation...")

            # Clean ground truth
            gt = None
            if ground_truth:
                gt = str(ground_truth).strip()
                if gt:
                    print(f"✅ Ground truth: {gt[:100]}...")
                else:
                    print(f"ℹ️ Ground truth: (empty)")

            try:
                evaluation_scores = evaluator.evaluate_single_query(
                    question=question,
                    answer=result['answer'],
                    contexts=contexts,
                    ground_truth=gt
                )

                if evaluation_scores:
                    print(f"\n✅ DeepEval evaluation completed!")
                    evaluation_report = evaluator.generate_report(evaluation_scores)

                    # Store evaluation result
                    app_state.evaluation_results.append({
                        'timestamp': datetime.now().isoformat(),
                        'question': question,
                        'answer': result['answer'],
                        'scores': evaluation_scores,
                        'ground_truth': gt
                    })
                else:
                    evaluation_report = """
⚠️ **Evaluation Failed**

DeepEval did not return scores. Check console output for details.

**Troubleshooting:**
1. Verify Cohere API key is valid and has quota
2. Check that contexts are not empty
3. Try a simpler question first
4. Review console for detailed error messages
"""

            except Exception as eval_error:
                print(f"\n❌ Evaluation exception: {str(eval_error)}")
                import traceback
                print(traceback.format_exc())
                evaluation_report = f"""
❌ **Evaluation Error**

```
{str(eval_error)}
```

**Check console for full traceback.**

**Quick fixes:**
1. Restart kernel and reload all cells
2. Verify Cohere API key: `{config.COHERE_API_KEY[:10]}...`
3. Check internet connection
4. Try without ground truth first
"""

        # Format answer for display
        answer_text = f"""
## 💡 Answer

{result['answer']}

---

### ⏱️ Performance Metrics

- **Retrieval Time:** {result['retrieval_time']:.3f}s
- **Reranking Time:** {result['rerank_time']:.3f}s
- **Generation Time:** {result['generation_time']:.3f}s
- **Total Time:** {result['total_time']:.3f}s
- **Documents Retrieved:** {len(contexts)}
"""

        # Add evaluation scores if available
        if evaluation_scores:
            answer_text += f"""

---

### 📊 DeepEval Scores

"""
            for metric, score in evaluation_scores.items():
                emoji = "✅" if score >= 0.7 else "⚠️" if score >= 0.5 else "❌"
                metric_name = metric.replace('_', ' ').title()

                # Special handling for hallucination (higher is better)
                if metric == 'hallucination':
                    answer_text += f"- {emoji} **{metric_name}:** {score:.4f} (higher = less hallucination)\n"
                else:
                    answer_text += f"- {emoji} **{metric_name}:** {score:.4f}\n"

        # Format sources
        sources_data = []
        for i, doc in enumerate(result['sources'], 1):
            sources_data.append({
                'Rank': i,
                'Source': doc.metadata.get('source', 'Unknown'),
                'Rerank Score': f"{doc.metadata.get('rerank_score', 0):.4f}",
                'Length': f"{len(doc.page_content)} chars",
                'Preview': doc.page_content[:150] + "..."
            })

        sources_df = pd.DataFrame(sources_data) if sources_data else None

        # Store in chat history
        query_id = len(app_state.chat_history)
        app_state.chat_history.append({
            'query_id': query_id,
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'question': question,
            'answer': result['answer'],
            'total_time': result['total_time'],
            'evaluation_scores': evaluation_scores,
            'contexts': contexts,
            'ground_truth': ground_truth if ground_truth and str(ground_truth).strip() else None
        })

        print(f"\n✅ Query complete! Query ID: {query_id}")
        print(f"{'='*70}\n")

        return answer_text, sources_df, evaluation_report, query_id, evaluation_scores

    except Exception as e:
        import traceback
        error_trace = traceback.format_exc()

        print(f"\n❌ QUERY ERROR:")
        print(error_trace)

        error_msg = f"""
## ❌ Query Error

**Error:** {str(e)}

**Traceback:**
```
{error_trace}
```

**Troubleshooting:**
1. System initialization: {'✅' if app_state.is_initialized else '❌'}
2. Check console output above
3. Verify Cohere API key
4. Try restarting kernel
"""
        return error_msg, None, "Error occurred", None, None




# CELL 5: Feedback Collection Functions


In [13]:
def record_thumbs_feedback(query_id, feedback_type):
    """Record thumbs up/down feedback"""

    if query_id is None or query_id >= len(app_state.chat_history):
        return "❌ Invalid query ID"

    app_state.add_feedback(
        query_id=query_id,
        feedback_type=feedback_type
    )

    return f"✅ Feedback recorded: {feedback_type}!"

def record_rating_feedback(query_id, rating, comment=""):
    """Record star rating feedback"""

    if query_id is None or query_id >= len(app_state.chat_history):
        return "❌ Invalid query ID"

    app_state.add_feedback(
        query_id=query_id,
        feedback_type='rating',
        rating=rating,
        comment=comment
    )

    return f"✅ Rating recorded: {rating}/5 stars!"

def record_correction(query_id, corrected_answer, comment=""):
    """Record corrected answer from user"""

    if query_id is None or query_id >= len(app_state.chat_history):
        return "❌ Invalid query ID"

    app_state.add_feedback(
        query_id=query_id,
        feedback_type='correction',
        corrected_answer=corrected_answer,
        comment=comment
    )

    # Add to ground truth
    app_state.ground_truth_qa.append({
        'question': app_state.chat_history[query_id]['question'],
        'ground_truth': corrected_answer,
        'contexts': app_state.chat_history[query_id].get('contexts', [])
    })

    return f"✅ Correction recorded and added to ground truth!"

def get_feedback_dashboard():
    """Generate feedback dashboard"""

    try:
        summary = app_state.get_feedback_summary()

        # Handle case when there's no feedback
        if isinstance(summary, str):
            return f"# 📊 Feedback Dashboard\n\n{summary}"

        dashboard = f"""
# 📊 Feedback Dashboard

## Overall Statistics

- **Total Feedback Received:** {summary['total_feedback']}
- **Thumbs Up:** 👍 {summary['thumbs_up']}
- **Thumbs Down:** 👎 {summary['thumbs_down']}
- **Corrections Provided:** ✏️ {summary['corrections']}
- **Average Rating:** ⭐ {summary['average_rating']:.2f}/5.0
- **Satisfaction Rate:** {summary['satisfaction_rate']:.1f}%

## Recent Feedback

"""

        # Show last 5 feedback items
        if app_state.feedback_data:
            for fb in app_state.feedback_data[-5:]:
                dashboard += f"""
---

**{fb['timestamp']}** - {fb['feedback_type']}

- **Question:** {fb['question'][:100] if fb['question'] else 'N/A'}...
"""
                if fb.get('rating'):
                    dashboard += f"- **Rating:** {'⭐' * fb['rating']}\n"
                if fb.get('comment'):
                    dashboard += f"- **Comment:** {fb['comment']}\n"
                if fb.get('corrected_answer'):
                    dashboard += f"- **Correction:** {fb['corrected_answer'][:100]}...\n"
        else:
            dashboard += "\n*No feedback items yet*\n"

        return dashboard

    except Exception as e:
        import traceback
        error_msg = f"""
# ❌ Dashboard Error

**Error:** {str(e)}

**Traceback:**
```
{traceback.format_exc()}
```

**Quick Fix:** Restart kernel and try again.
"""
        return error_msg

def export_all_data():
    """Export all feedback and evaluation data"""

    try:
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

        # Export feedback
        feedback_file = f"feedback_{timestamp}.json"
        with open(feedback_file, 'w', encoding='utf-8') as f:
            json.dump(app_state.feedback_data, f, indent=2, ensure_ascii=False)

        # Export evaluation results
        eval_file = f"evaluation_{timestamp}.json"
        with open(eval_file, 'w', encoding='utf-8') as f:
            json.dump(app_state.evaluation_results, f, indent=2, ensure_ascii=False)

        # Export ground truth
        gt_file = f"ground_truth_{timestamp}.json"
        with open(gt_file, 'w', encoding='utf-8') as f:
            json.dump(app_state.ground_truth_qa, f, indent=2, ensure_ascii=False)

        summary = f"""
✅ **Data Exported Successfully!**

**Files created:**
1. `{feedback_file}` - User feedback data
2. `{eval_file}` - DeepEval evaluation results
3. `{gt_file}` - Ground truth Q&A pairs

**Total records:**
- Feedback: {len(app_state.feedback_data)}
- Evaluations: {len(app_state.evaluation_results)}
- Ground Truth: {len(app_state.ground_truth_qa)}

**Download the files using the file widgets below.**
"""

        return summary, feedback_file, eval_file, gt_file

    except Exception as e:
        import traceback
        error_msg = f"""
❌ **Export Failed**

**Error:** {str(e)}
```
{traceback.format_exc()}
```
"""
        return error_msg, None, None, None



# CELL 6: Batch Evaluation Function

In [14]:
def run_batch_evaluation():
    """Run batch evaluation using DeepEval"""

    print(f"\n{'='*70}")
    print(f"🔬 BATCH DEEPEVAL EVALUATION")
    print(f"{'='*70}")

    # Check chat history
    print(f"\n📊 Checking chat history...")
    print(f"   Total queries: {len(app_state.chat_history)}")

    # Find queries with ground truth
    queries_with_gt = []
    for i, query in enumerate(app_state.chat_history):
        gt = query.get('ground_truth')
        if gt and str(gt).strip():
            print(f"   ✅ Query {i}: Has ground truth")
            queries_with_gt.append(query)
        else:
            print(f"   ⚠️ Query {i}: No ground truth")

    print(f"\n📈 Summary:")
    print(f"   Total queries: {len(app_state.chat_history)}")
    print(f"   With ground truth: {len(queries_with_gt)}")

    if len(queries_with_gt) == 0:
        return f"""
❌ **No Ground Truth Data Available**

**Current Status:**
- Total queries in history: {len(app_state.chat_history)}
- Queries with ground truth: 0

**How to fix this:**

### Option 1: Evaluate Queries with Ground Truth
1. Go to **"Ask & Evaluate"** tab
2. Enter a question (e.g., "MFCC signifie quoi?")
3. Check **✓ "Evaluate with DeepEval"**
4. Enter ground truth: "MFCC est l'acronyme de Mel Frequency Cepstral Coefficients"
5. Click **"Get Answer"**
6. Repeat 3-5 times with different questions
7. Return here and click **"Run Batch Evaluation"**

### Option 2: Provide Corrections
1. Ask some questions normally (without evaluation)
2. For wrong answers, use **"Provide Correction"** section
3. Enter the correct answer
4. Submit 3-5 corrections
5. Return here and click **"Run Batch Evaluation"**

**Tip:** You need at least 1 query with ground truth to run batch evaluation.

**Debug Info:**
- Queries evaluated: {len([q for q in app_state.chat_history if q.get('evaluation_scores')])}
- Corrections provided: {len([q for q in app_state.chat_history if q.get('ground_truth')])}
""", None

    try:
        print(f"\n📝 Preparing {len(queries_with_gt)} queries for batch evaluation...")

        # Prepare Q&A pairs
        qa_pairs = []
        for i, query in enumerate(queries_with_gt, 1):
            print(f"\n   [{i}/{len(queries_with_gt)}]")
            print(f"   Question: {query['question'][:60]}...")
            print(f"   Ground truth: {query['ground_truth'][:60]}...")

            # Get contexts
            if 'contexts' in query and query['contexts']:
                contexts = query['contexts']
                print(f"   ✅ Using stored contexts ({len(contexts)})")
            else:
                print(f"   ⏳ Re-querying to get fresh contexts...")
                try:
                    result = app_state.rag_pipeline.query(query['question'])
                    contexts = [doc.page_content for doc in result['sources']]
                    query['answer'] = result['answer']
                    print(f"   ✅ Got {len(contexts)} fresh contexts")
                except Exception as e:
                    print(f"   ❌ Re-query failed: {str(e)[:100]}")
                    continue

            # Validate contexts
            contexts = [str(c).strip() for c in contexts if c and str(c).strip()]

            if not contexts:
                print(f"   ❌ Skipping: no valid contexts")
                continue

            # Add to batch
            qa_pairs.append({
                'question': query['question'],
                'answer': query['answer'],
                'contexts': contexts,
                'ground_truth': query['ground_truth']
            })
            print(f"   ✅ Added to batch")

        if not qa_pairs:
            return """
❌ **No Valid Queries for Batch Evaluation**

All queries had issues (missing contexts, answers, etc.)

**Try:**
1. Submit 2-3 new queries with evaluation enabled
2. Make sure to provide ground truth answers
3. Click "Get Answer" and wait for results
4. Then run batch evaluation again
""", None

        print(f"\n{'='*70}")
        print(f"⏳ Running DeepEval on {len(qa_pairs)} queries...")
        print(f"{'='*70}")

        # Run batch evaluation using the evaluator
        scores = evaluator.evaluate_batch(qa_pairs)

        if scores and len(scores) > 0:
            # Generate report
            timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
            report_file = f"deepeval_batch_{timestamp}.txt"
            report = evaluator.generate_report(scores, report_file)

            # Add header with details
            header = f"""
# 🎯 DeepEval Batch Evaluation Results

**Date:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
**Queries Evaluated:** {len(qa_pairs)}
**Total in History:** {len(app_state.chat_history)}
**Evaluation Tool:** DeepEval

---

"""
            final_report = header + report

            print(f"\n{'='*70}")
            print(f"✅ BATCH EVALUATION COMPLETE!")
            print(f"{'='*70}")
            print(f"📄 Report saved: {report_file}")
            print(f"📊 Queries evaluated: {len(qa_pairs)}")
            print(f"\n📈 Results:")
            for metric, score in scores.items():
                emoji = "✅" if score >= 0.7 else "⚠️" if score >= 0.5 else "❌"
                print(f"   {emoji} {metric}: {score:.4f}")
            print(f"{'='*70}\n")

            return final_report, report_file
        else:
            return f"""
❌ **Batch Evaluation Failed**

DeepEval did not produce scores.

**What happened:**
- Prepared {len(qa_pairs)} queries
- All had ground truth
- But evaluation returned no scores

**Troubleshooting:**
1. Check console output above for errors
2. Verify Cohere API is working (check usage)
3. Try evaluating just 1 query first
4. Wait 5 minutes and try again (rate limits)

**Debug:**
- Queries prepared: {len(qa_pairs)}
- Scores returned: {scores}
""", None

    except Exception as e:
        import traceback
        error_trace = traceback.format_exc()

        print(f"\n{'='*70}")
        print(f"❌ BATCH EVALUATION ERROR")
        print(f"{'='*70}")
        print(error_trace)
        print(f"{'='*70}\n")

        return f"""
❌ **Batch Evaluation Error**

**Error:** {str(e)}

**Traceback:**
```
{error_trace[:500]}
```

**Common Causes:**
1. **API Rate Limit:** Wait 5-10 minutes
2. **No Ground Truth:** Provide ground truth in queries
3. **Empty Contexts:** Documents not loaded properly
4. **API Key Issue:** Check Cohere dashboard

**Quick Fix:**
1. Check console output above for detailed error
2. Try evaluating a single query first
3. Verify ground truth is provided
4. Make sure contexts are not empty

**Need Help?**
Share the full error from console for debugging.
""", None


print("✅ Fixed batch evaluation function loaded!")
print("\n💡 To use batch evaluation:")
print("   1. Submit 3-5 queries with ground truth")
print("   2. Go to 'Batch Evaluation' tab")
print("   3. Click 'Run Batch Evaluation'")
print("\n🔍 The function will:")
print("   • Find all queries with ground truth")
print("   • Re-evaluate them with DeepEval")
print("   • Show average scores across all queries")
print("   • Generate a comprehensive report")

✅ Fixed batch evaluation function loaded!

💡 To use batch evaluation:
   1. Submit 3-5 queries with ground truth
   2. Go to 'Batch Evaluation' tab
   3. Click 'Run Batch Evaluation'

🔍 The function will:
   • Find all queries with ground truth
   • Re-evaluate them with DeepEval
   • Show average scores across all queries
   • Generate a comprehensive report


#CELL 7: helper : ADD GROUND TRUTH TO EXISTING QUERIES

In [15]:
def add_ground_truth_to_query(query_id, ground_truth):
    """Add ground truth to an existing query"""

    if query_id < 0 or query_id >= len(app_state.chat_history):
        return f"❌ Invalid query ID. Must be between 0 and {len(app_state.chat_history)-1}"

    query = app_state.chat_history[query_id]
    query['ground_truth'] = ground_truth

    return f"""
✅ **Ground Truth Added!**

**Query ID:** {query_id}
**Question:** {query['question'][:100]}...
**Ground Truth:** {ground_truth}

Now you can run batch evaluation!
"""


def show_queries_without_ground_truth():
    """Show all queries that don't have ground truth"""

    print("="*70)
    print("📋 QUERIES WITHOUT GROUND TRUTH")
    print("="*70)

    queries_without_gt = []

    for i, query in enumerate(app_state.chat_history):
        if not query.get('ground_truth') or not str(query.get('ground_truth', '')).strip():
            queries_without_gt.append((i, query))

    if not queries_without_gt:
        print("\n✅ All queries have ground truth!")
        print(f"   Total queries: {len(app_state.chat_history)}")
        print(f"   Ready for batch evaluation!")
        return

    print(f"\nFound {len(queries_without_gt)} queries without ground truth:\n")

    for query_id, query in queries_without_gt:
        print(f"Query ID: {query_id}")
        print(f"  Question: {query['question'][:80]}...")
        print(f"  Answer: {query['answer'][:80]}...")
        print(f"  To add ground truth, use:")
        print(f"    add_ground_truth_to_query({query_id}, 'your ground truth here')")
        print()

    print("="*70)


def prepare_for_batch_evaluation():
    """Quick way to prepare queries for batch evaluation"""

    print("="*70)
    print("🎯 BATCH EVALUATION PREPARATION")
    print("="*70)

    total = len(app_state.chat_history)
    with_gt = len([q for q in app_state.chat_history if q.get('ground_truth')])

    print(f"\n📊 Current Status:")
    print(f"   Total queries: {total}")
    print(f"   With ground truth: {with_gt}")
    print(f"   Without ground truth: {total - with_gt}")

    if with_gt == 0:
        print(f"\n⚠️ NO GROUND TRUTH DATA")
        print(f"\n💡 Two options:")
        print(f"\n1️⃣ Submit new queries with ground truth:")
        print(f"   • Go to 'Ask & Evaluate' tab")
        print(f"   • Check 'Evaluate with DeepEval'")
        print(f"   • Enter ground truth answer")
        print(f"   • Submit 3-5 queries")
        print(f"\n2️⃣ Add ground truth to existing queries:")
        print(f"   • Run: show_queries_without_ground_truth()")
        print(f"   • For each query, run:")
        print(f"     add_ground_truth_to_query(query_id, 'correct answer')")

    elif with_gt < 3:
        print(f"\n⚠️ NEED MORE DATA")
        print(f"   You have {with_gt} queries with ground truth")
        print(f"   Recommended: At least 3-5 queries")
        print(f"\n💡 Add {3 - with_gt} more queries with ground truth")

    else:
        print(f"\n✅ READY FOR BATCH EVALUATION!")
        print(f"   You have {with_gt} queries with ground truth")
        print(f"   Go to 'Batch Evaluation' tab and click 'Run Batch Evaluation'")

    print("="*70)


# Example usage
print("="*70)
print("✅ GROUND TRUTH HELPERS LOADED!")
print("="*70)
print("\n📚 Available Functions:")
print("\n1️⃣ show_queries_without_ground_truth()")
print("   Shows all queries that need ground truth")
print("\n2️⃣ add_ground_truth_to_query(query_id, 'ground truth text')")
print("   Adds ground truth to a specific query")
print("\n3️⃣ prepare_for_batch_evaluation()")
print("   Shows status and next steps")
print("\n💡 Quick Start:")
print("   >>> prepare_for_batch_evaluation()")
print("="*70)

# Auto-check status
print("\n🔍 Auto-checking current status...")
prepare_for_batch_evaluation()

✅ GROUND TRUTH HELPERS LOADED!

📚 Available Functions:

1️⃣ show_queries_without_ground_truth()
   Shows all queries that need ground truth

2️⃣ add_ground_truth_to_query(query_id, 'ground truth text')
   Adds ground truth to a specific query

3️⃣ prepare_for_batch_evaluation()
   Shows status and next steps

💡 Quick Start:
   >>> prepare_for_batch_evaluation()

🔍 Auto-checking current status...
🎯 BATCH EVALUATION PREPARATION

📊 Current Status:
   Total queries: 0
   With ground truth: 0
   Without ground truth: 0

⚠️ NO GROUND TRUTH DATA

💡 Two options:

1️⃣ Submit new queries with ground truth:
   • Go to 'Ask & Evaluate' tab
   • Check 'Evaluate with DeepEval'
   • Enter ground truth answer
   • Submit 3-5 queries

2️⃣ Add ground truth to existing queries:
   • Run: show_queries_without_ground_truth()
   • For each query, run:
     add_ground_truth_to_query(query_id, 'correct answer')


# =============================================================================
# SECTION 10: MAIN EXECUTION
# ==========================================================

#==========================================================================================================

# CELL 1: HELPER FUNCTIONS FOR GRADIO INTERFACE


In [16]:
# =============================================================================
# SECTION 10 - CELL 1: FIXED HELPER FUNCTIONS FOR GRADIO INTERFACE
# Replace your entire helper functions cell with this
# =============================================================================

import os
import shutil
import pandas as pd
from datetime import datetime

def list_files(folder_path):
    """List all supported files in a folder"""

    if not os.path.exists(folder_path):
        return f"❌ Folder not found: {folder_path}", None

    supported_formats = ['.pdf', '.docx', '.txt', '.md']
    files_found = []

    try:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                if any(file.endswith(ext) for ext in supported_formats):
                    file_path = os.path.join(root, file)
                    file_size = os.path.getsize(file_path) / 1024  # KB

                    files_found.append({
                        'File Name': file,
                        'Type': os.path.splitext(file)[1],
                        'Size (KB)': f"{file_size:.2f}",
                        'Path': file_path
                    })

        if not files_found:
            return f"📂 No supported files found in: {folder_path}", None

        df = pd.DataFrame(files_found)
        message = f"✅ Found {len(files_found)} files in: {folder_path}"

        # Store file paths in app_state
        app_state.file_paths = [item['Path'] for item in files_found]

        return message, df

    except Exception as e:
        return f"❌ Error: {str(e)}", None


def upload_and_list(uploaded_files, target_folder):
    """Handle file uploads and save to target folder"""

    if uploaded_files is None or len(uploaded_files) == 0:
        return "❌ No files uploaded", None

    os.makedirs(target_folder, exist_ok=True)

    saved_files = []

    try:
        for file in uploaded_files:
            # Copy file to target folder
            dest_path = os.path.join(target_folder, os.path.basename(file))
            shutil.copy2(file, dest_path)

            file_size = os.path.getsize(dest_path) / 1024  # KB

            saved_files.append({
                'File Name': os.path.basename(file),
                'Type': os.path.splitext(file)[1],
                'Size (KB)': f"{file_size:.2f}",
                'Path': dest_path
            })

        # Store paths
        app_state.file_paths = [item['Path'] for item in saved_files]

        df = pd.DataFrame(saved_files)
        message = f"✅ Uploaded {len(saved_files)} files to: {target_folder}"

        return message, df

    except Exception as e:
        return f"❌ Upload error: {str(e)}", None


def initialize_from_files():
    """Initialize RAG system from stored file paths"""

    if not app_state.file_paths:
        return "❌ No files selected. Please browse or upload files first.", "🔴 **Offline**", None

    try:
        print("\n" + "="*70)
        print("🚀 INITIALIZING RAG SYSTEM")
        print("="*70)

        # Initialize pipeline
        app_state.rag_pipeline = RAGPipeline()

        # Ingest documents
        app_state.rag_pipeline.ingest_documents(app_state.file_paths)

        # Update state
        app_state.is_initialized = True
        app_state.documents_loaded = len(app_state.file_paths)

        # Get statistics
        stats = get_system_statistics()

        message = f"""
## ✅ System Initialized Successfully!

**Documents Loaded:** {app_state.documents_loaded}
**Status:** 🟢 Online and Ready

You can now ask questions in the "Ask & Evaluate" tab.
"""

        return message, "🟢 **Online**", stats

    except Exception as e:
        import traceback
        error_msg = f"""
## ❌ Initialization Failed

**Error:** {str(e)}

**Details:**
```
{traceback.format_exc()}
```
"""
        return error_msg, "🔴 **Offline**", None


def get_system_statistics():
    """Get system statistics as DataFrame"""

    try:
        if not app_state.is_initialized:
            return pd.DataFrame({
                'Metric': ['System Status'],
                'Value': ['Not Initialized']
            })

        # Safely get total_chunks
        total_chunks = getattr(app_state, 'total_chunks', 0)

        stats = {
            'Metric': [
                'Documents Loaded',
                'Total Chunks',
                'Queries Processed',
                'Feedback Received',
                'Evaluations Run',
                'Ground Truth Pairs'
            ],
            'Value': [
                app_state.documents_loaded,
                total_chunks,
                len(app_state.chat_history),
                len(app_state.feedback_data),
                len(app_state.evaluation_results),
                len(app_state.ground_truth_qa)
            ]
        }

        return pd.DataFrame(stats)

    except Exception as e:
        print(f"Statistics error: {e}")
        return pd.DataFrame({
            'Metric': ['Error'],
            'Value': [str(e)]
        })


def format_chat_history():
    """Format chat history for display"""

    try:
        if not app_state.chat_history:
            return "📝 No queries yet. Start asking questions in the 'Ask & Evaluate' tab!"

        history_md = "# 📜 Query History\n\n"

        for i, entry in enumerate(reversed(app_state.chat_history), 1):
            # Safely get values
            query_id = entry.get('query_id', 'N/A')
            timestamp = entry.get('timestamp', 'N/A')
            question = entry.get('question', 'N/A')
            answer = entry.get('answer', 'N/A')
            total_time = entry.get('total_time', 0)

            history_md += f"""
---

### Query #{query_id} - {timestamp}

**❓ Question:**
{question}

**💡 Answer:**
{answer[:300]}...

**⏱️ Response Time:** {total_time:.3f}s

"""

            # Add evaluation scores if available
            if entry.get('evaluation_scores'):
                scores = entry['evaluation_scores']
                history_md += f"""
**📊 Evaluation Scores:**
"""
                for metric, score in scores.items():
                    history_md += f"- **{metric.replace('_', ' ').title()}:** {score:.3f}\n"
                history_md += "\n"

        return history_md

    except Exception as e:
        import traceback
        error_msg = f"""
# ❌ History Error

**Error:** {str(e)}

**Traceback:**
```
{traceback.format_exc()}
```
"""
        return error_msg


def clear_history():
    """Clear chat history"""

    count = len(app_state.chat_history)
    app_state.chat_history = []

    return f"✅ Cleared {count} queries from history", "📝 No queries yet. Start asking questions!"


def get_system_info():
    """Get system information"""

    try:
        # Safely get total_chunks
        total_chunks = getattr(app_state, 'total_chunks', 0)

        info = f"""
# ℹ️ System Information

## 📊 Status
- **System:** {'🟢 Online' if app_state.is_initialized else '🔴 Offline'}
- **Documents:** {app_state.documents_loaded}
- **Chunks:** {total_chunks}
- **Queries:** {len(app_state.chat_history)}
- **Feedback:** {len(app_state.feedback_data)}

## ⚙️ Configuration
- **Model:** {config.COHERE_MODEL}
- **Embedding Model:** {config.COHERE_EMBEDDING_MODEL}
- **Rerank Model:** {config.COHERE_RERANK_MODEL}
- **Chunk Size:** {config.CHUNK_SIZE}
- **Chunk Overlap:** {config.CHUNK_OVERLAP}
- **Top K Retrieval:** {config.TOP_K}
- **Top K Rerank:** {config.TOP_K_RERANK}

## 📈 Performance
"""

        if app_state.chat_history:
            avg_time = sum(q.get('total_time', 0) for q in app_state.chat_history) / len(app_state.chat_history)
            times = [q.get('total_time', 0) for q in app_state.chat_history]
            info += f"""
- **Average Response Time:** {avg_time:.3f}s
- **Fastest Query:** {min(times):.3f}s
- **Slowest Query:** {max(times):.3f}s
"""
        else:
            info += "\n- No queries processed yet\n"

        # Feedback summary
        if app_state.feedback_data:
            summary = app_state.get_feedback_summary()
            if isinstance(summary, dict):
                info += f"""
## 💭 Feedback Summary
- **Satisfaction Rate:** {summary['satisfaction_rate']:.1f}%
- **Average Rating:** {summary['average_rating']:.2f}/5.0
- **Thumbs Up:** {summary['thumbs_up']}
- **Thumbs Down:** {summary['thumbs_down']}
- **Corrections:** {summary['corrections']}
"""

        return info

    except Exception as e:
        import traceback
        return f"""
# ❌ System Info Error

**Error:** {str(e)}

```
{traceback.format_exc()}
```
"""


print("✅ Fixed helper functions loaded successfully!")
print("\nAvailable functions:")
print("  ✅ list_files()")
print("  ✅ upload_and_list()")
print("  ✅ initialize_from_files()")
print("  ✅ get_system_statistics()")
print("  ✅ format_chat_history()")
print("  ✅ clear_history()")
print("  ✅ get_system_info()")

✅ Fixed helper functions loaded successfully!

Available functions:
  ✅ list_files()
  ✅ upload_and_list()
  ✅ initialize_from_files()
  ✅ get_system_statistics()
  ✅ format_chat_history()
  ✅ clear_history()
  ✅ get_system_info()


# CELL 2: FINAL COMPLETE GRADIO INTERFACE WITH DEEPEVAL


In [17]:
!pip install gradio --upgrade --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.0/23.0 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 3.3 MB/s eta 0:00:00


In [18]:
import gradio as gr

# Create interface
demo = gr.Blocks(title="RAG System with DeepEval & Human Feedback")

with demo:

    # Header
    gr.HTML("""
    <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                color: white; padding: 2rem; border-radius: 10px;
                text-align: center; margin-bottom: 2rem;
                box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);">
        <h1>🚀 Enhanced RAG System</h1>
        <p style="font-size: 1.2rem;">With DeepEval Evaluation & Human-in-the-Loop Feedback</p>
        <p style="font-size: 0.9rem; opacity: 0.9;">Powered by Cohere • FAISS • LangChain • DeepEval</p>
    </div>
    """)

    # Hidden state for current query ID
    current_query_id = gr.State(None)
    current_eval_scores = gr.State(None)

    with gr.Tabs():

        # =====================================================================
        # TAB 1: DOCUMENT SETUP
        # =====================================================================
        with gr.Tab("📂 Document Setup"):
            gr.Markdown("""
            ## 📂 Load Your Documents

            Upload or browse for documents to build your knowledge base.
            """)

            with gr.Row():
                with gr.Column(scale=3):
                    folder_input = gr.Textbox(
                        label="📂 Folder Path",
                        value="/content/",
                        placeholder="/content/your-folder"
                    )
                with gr.Column(scale=1):
                    browse_btn = gr.Button("🔍 Browse", variant="secondary")

            browse_output = gr.Markdown()
            files_table = gr.Dataframe(label="📄 Available Files")

            browse_btn.click(
                fn=list_files,
                inputs=[folder_input],
                outputs=[browse_output, files_table]
            )

            gr.Markdown("---")

            gr.Markdown("### 📤 Or Upload Files Directly")

            upload_files = gr.File(
                label="Select Files to Upload",
                file_count="multiple",
                type="filepath"
            )
            upload_btn = gr.Button("📤 Upload Files", variant="secondary")

            upload_btn.click(
                fn=upload_and_list,
                inputs=[upload_files, folder_input],
                outputs=[browse_output, files_table]
            )

            gr.Markdown("---")
            gr.Markdown("### 🚀 Initialize RAG System")

            load_btn = gr.Button("🚀 Initialize System", variant="primary", size="lg")

            with gr.Row():
                with gr.Column():
                    load_output = gr.Markdown()
                with gr.Column():
                    system_status = gr.Markdown("🔴 **Offline**")

            stats_table = gr.Dataframe(label="📊 System Statistics")

            load_btn.click(
                fn=initialize_from_files,
                inputs=[],
                outputs=[load_output, system_status, stats_table]
            )

        # =====================================================================
        # TAB 2: ASK & EVALUATE WITH DEEPEVAL
        # =====================================================================
        with gr.Tab("💬 Ask & Evaluate"):
            gr.Markdown("""
            ## 💬 Query with DeepEval Evaluation

            Ask questions and optionally evaluate responses with DeepEval metrics.
            """)

            with gr.Row():
                with gr.Column(scale=2):
                    question_input = gr.Textbox(
                        label="❓ Your Question",
                        placeholder="Example: MFCC signifie quoi?",
                        lines=3
                    )

                with gr.Column(scale=1):
                    use_reranking = gr.Checkbox(
                        label="🎯 Enable Reranking",
                        value=True,
                        info="Use Cohere reranking"
                    )
                    top_k = gr.Slider(
                        label="📊 Top-K Results",
                        minimum=1,
                        maximum=10,
                        value=5,
                        step=1,
                        info="Number of documents to retrieve"
                    )

                    evaluate_checkbox = gr.Checkbox(
                        label="🔬 Evaluate with DeepEval",
                        value=False,
                        info="Run DeepEval metrics (takes 10-20s)"
                    )

            ground_truth_input = gr.Textbox(
                label="✅ Ground Truth (Optional - for full evaluation)",
                placeholder="Example: MFCC est l'acronyme de Mel Frequency Cepstral Coefficients",
                lines=2,
                visible=False,
                info="Provide expected answer for complete metrics (Contextual Precision & Recall)"
            )

            # Show/hide ground truth input
            evaluate_checkbox.change(
                fn=lambda x: gr.update(visible=x),
                inputs=[evaluate_checkbox],
                outputs=[ground_truth_input]
            )

            query_btn = gr.Button("🔍 Get Answer", variant="primary", size="lg")

            # Answer display
            answer_output = gr.Markdown(label="💡 Answer")

            # DeepEval Evaluation Report
            with gr.Accordion("📊 DeepEval Evaluation Report", open=False):
                gr.Markdown("""
                **DeepEval Metrics Explained:**

                | Metric | Description | Ground Truth? |
                |--------|-------------|---------------|
                | **Faithfulness** | Answer is factually grounded in retrieved context | ❌ No |
                | **Answer Relevancy** | Answer directly addresses the question | ❌ No |
                | **Contextual Precision** | Retrieved documents are relevant to question | ✅ Yes |
                | **Contextual Recall** | All relevant information was retrieved | ✅ Yes |

                **Score Interpretation:**
                - 🏆 **0.8-1.0**: Excellent
                - ✅ **0.7-0.8**: Good
                - ⚠️ **0.5-0.7**: Fair
                - ❌ **Below 0.5**: Needs improvement
                """)
                evaluation_report = gr.Markdown("Run evaluation to see detailed results...")

            # Sources
            with gr.Accordion("📚 Source Documents", open=True):
                sources_output = gr.Dataframe(label="Retrieved Sources")

            gr.Markdown("---")

            # FEEDBACK SECTION
            gr.Markdown("### 💭 Provide Feedback on This Answer")
            gr.Markdown("Help improve the system by providing feedback!")

            with gr.Row():
                thumbs_up_btn = gr.Button("👍 Helpful", size="sm", variant="secondary")
                thumbs_down_btn = gr.Button("👎 Not Helpful", size="sm", variant="secondary")

            with gr.Row():
                with gr.Column():
                    rating_slider = gr.Slider(
                        label="⭐ Rate Answer Quality (1-5 stars)",
                        minimum=1,
                        maximum=5,
                        value=3,
                        step=1
                    )
                with gr.Column():
                    rating_btn = gr.Button("📝 Submit Rating", variant="secondary")

            with gr.Accordion("✏️ Provide Correction (Builds Ground Truth)", open=False):
                gr.Markdown("""
                **Important:** When you provide a correction, it's automatically added as ground truth
                for batch evaluation!
                """)
                correction_input = gr.Textbox(
                    label="✅ Correct Answer",
                    placeholder="Enter the correct answer here...",
                    lines=4
                )
                correction_comment = gr.Textbox(
                    label="💬 Additional Comments (Optional)",
                    placeholder="Why was the original answer incorrect?",
                    lines=2
                )
                correction_btn = gr.Button("✅ Submit Correction", variant="primary")

            feedback_status = gr.Markdown()

            # Query execution
            query_btn.click(
                fn=query_with_evaluation,
                inputs=[question_input, use_reranking, top_k, evaluate_checkbox, ground_truth_input],
                outputs=[answer_output, sources_output, evaluation_report, current_query_id, current_eval_scores]
            )

            # Feedback buttons
            thumbs_up_btn.click(
                fn=lambda qid: record_thumbs_feedback(qid, 'thumbs_up'),
                inputs=[current_query_id],
                outputs=[feedback_status]
            )

            thumbs_down_btn.click(
                fn=lambda qid: record_thumbs_feedback(qid, 'thumbs_down'),
                inputs=[current_query_id],
                outputs=[feedback_status]
            )

            rating_btn.click(
                fn=record_rating_feedback,
                inputs=[current_query_id, rating_slider],
                outputs=[feedback_status]
            )

            correction_btn.click(
                fn=record_correction,
                inputs=[current_query_id, correction_input, correction_comment],
                outputs=[feedback_status]
            )

            # Example questions
            gr.Markdown("---")
            gr.Markdown("### 💡 Example Questions")
            gr.Examples(
                examples=[
                    ["MFCC signifie quoi?"],
                    ["Qu'est-ce que le spectrogramme?"],
                    ["Comment calculer les MFCC?"],
                    ["Quelle est l'utilité des MFCC?"],
                    ["What are the main topics in these documents?"]
                ],
                inputs=question_input
            )

        # =====================================================================
        # TAB 3: FEEDBACK DASHBOARD
        # =====================================================================
        with gr.Tab("📊 Feedback Dashboard"):
            gr.Markdown("""
            ## 📊 Human Feedback Analytics

            Monitor user feedback and system performance over time.
            """)

            refresh_dashboard_btn = gr.Button("🔄 Refresh Dashboard", variant="primary")
            dashboard_output = gr.Markdown("Click 'Refresh Dashboard' to see statistics...")

            refresh_dashboard_btn.click(
                fn=get_feedback_dashboard,
                inputs=[],
                outputs=[dashboard_output]
            )

            gr.Markdown("---")

            gr.Markdown("### 💾 Export Data for Analysis")
            gr.Markdown("Download all feedback, evaluation results, and ground truth data.")

            export_btn = gr.Button("📥 Export All Data", variant="secondary", size="lg")
            export_status = gr.Markdown()

            with gr.Row():
                feedback_file = gr.File(label="📋 Feedback Data (JSON)")
                eval_file = gr.File(label="📊 Evaluation Results (JSON)")
                gt_file = gr.File(label="✅ Ground Truth (JSON)")

            export_btn.click(
                fn=export_all_data,
                inputs=[],
                outputs=[export_status, feedback_file, eval_file, gt_file]
            )

        # =====================================================================
        # TAB 4: BATCH EVALUATION WITH DEEPEVAL
        # =====================================================================
        with gr.Tab("🔬 Batch Evaluation"):
            gr.Markdown("""
            ## 🔬 DeepEval Batch Evaluation

            Run comprehensive evaluation on all queries with ground truth.
            """)

            gr.Markdown("""
            ### 📖 How Batch Evaluation Works

            **Step 1:** Collect queries with ground truth
            - Option A: Use "Evaluate with DeepEval" checkbox when asking questions
            - Option B: Provide corrections using "Provide Correction" button

            **Step 2:** Click "Run Batch Evaluation" below

            **Step 3:** System evaluates all queries and calculates average scores

            **Step 4:** Get comprehensive report with recommendations

            ---

            **Minimum Requirement:** At least 1 query with ground truth
            **Recommended:** 3-5 queries for meaningful statistics
            """)

            # Status check
            with gr.Row():
                check_status_btn = gr.Button("🔍 Check Readiness", variant="secondary")
                batch_status_output = gr.Markdown()

            check_status_btn.click(
                fn=lambda: prepare_for_batch_evaluation() if 'prepare_for_batch_evaluation' in globals() else "Run helper functions cell first",
                inputs=[],
                outputs=[batch_status_output]
            )

            gr.Markdown("---")

            batch_eval_btn = gr.Button("🚀 Run Batch Evaluation", variant="primary", size="lg")

            batch_eval_output = gr.Markdown("Click 'Run Batch Evaluation' to start...")
            batch_eval_file = gr.File(label="📄 Download Evaluation Report")

            batch_eval_btn.click(
                fn=run_batch_evaluation,
                inputs=[],
                outputs=[batch_eval_output, batch_eval_file]
            )

            gr.Markdown("---")

            gr.Markdown("""
            ### 📊 DeepEval Metrics Reference

            | Metric | What It Measures | Good Score | Needs Ground Truth? |
            |--------|------------------|------------|---------------------|
            | **Faithfulness** | Answer is factually consistent with retrieved context | ≥ 0.7 | ❌ No |
            | **Answer Relevancy** | Answer directly addresses the user's question | ≥ 0.7 | ❌ No |
            | **Contextual Precision** | Retrieved documents are relevant and useful | ≥ 0.7 | ✅ Yes |
            | **Contextual Recall** | All relevant information was successfully retrieved | ≥ 0.7 | ✅ Yes |

            ### 🎯 Score Interpretation

            - **🏆 0.8 - 1.0**: Excellent - System is performing very well
            - **✅ 0.7 - 0.8**: Good - System is performing adequately
            - **⚠️ 0.5 - 0.7**: Fair - System needs some improvement
            - **❌ Below 0.5**: Poor - Significant improvements needed

            ### 💡 Tips for Better Evaluations

            1. **Provide accurate ground truth** - The quality of evaluation depends on good ground truth
            2. **Use diverse questions** - Test different types of queries
            3. **Evaluate regularly** - Run batch evaluation after every 10-20 queries
            4. **Compare over time** - Track if your system is improving
            5. **Focus on weak areas** - If a metric is low, investigate why
            """)

        # =====================================================================
        # TAB 5: QUERY HISTORY
        # =====================================================================
        with gr.Tab("📜 History"):
            gr.Markdown("## 📜 Query History & Management")

            with gr.Row():
                refresh_history_btn = gr.Button("🔄 Refresh History", variant="secondary")
                clear_history_btn = gr.Button("🗑️ Clear History", variant="stop")

            history_display = gr.Markdown("No queries yet. Start asking questions in the 'Ask & Evaluate' tab!")
            clear_status = gr.Markdown()

            refresh_history_btn.click(
                fn=format_chat_history,
                inputs=[],
                outputs=[history_display]
            )

            clear_history_btn.click(
                fn=clear_history,
                inputs=[],
                outputs=[clear_status, history_display]
            )

            gr.Markdown("---")

            gr.Markdown("## ℹ️ System Information")
            info_btn = gr.Button("📊 Show System Info", variant="secondary")
            info_display = gr.Markdown()

            info_btn.click(
                fn=get_system_info,
                inputs=[],
                outputs=[info_display]
            )

        # =====================================================================
        # TAB 6: HELP & DOCUMENTATION
        # =====================================================================
        with gr.Tab("ℹ️ Help"):
            gr.Markdown("""
            # 📖 Complete User Guide

            ## 🚀 Quick Start (5 Minutes)

            ### Step 1: Setup Your Documents (2 min)
            1. Go to **"📂 Document Setup"** tab
            2. Click **"🔍 Browse"** or use **"📤 Upload Files"**
            3. Click **"🚀 Initialize System"**
            4. Wait for "✅ System Initialized Successfully!"

            ### Step 2: Ask Your First Question (1 min)
            1. Go to **"💬 Ask & Evaluate"** tab
            2. Type a question (e.g., "MFCC signifie quoi?")
            3. Click **"🔍 Get Answer"**
            4. Review answer and sources

            ### Step 3: Try Evaluation (2 min)
            1. Check **"✓ Evaluate with DeepEval"**
            2. Enter ground truth answer
            3. Click **"🔍 Get Answer"**
            4. View evaluation scores in the report

            ---

            ## 📊 Understanding DeepEval Metrics

            ### Faithfulness (Always Available)
            **Question:** Is the answer based only on information from the documents?

            **What it checks:**
            - No hallucinations or made-up facts
            - All claims are supported by retrieved context
            - Answer doesn't contradict the documents

            **Example:**
            - ✅ Good: "MFCC stands for Mel Frequency Cepstral Coefficients" (if this is in the docs)
            - ❌ Bad: "MFCC was invented in 1980 by John Smith" (if this isn't in the docs)

            ### Answer Relevancy (Always Available)
            **Question:** Does the answer actually address what was asked?

            **What it checks:**
            - Answer is on-topic
            - No unnecessary information
            - Directly responds to the question

            **Example for "What is MFCC?":**
            - ✅ Good: "MFCC stands for Mel Frequency Cepstral Coefficients, used in audio processing"
            - ❌ Bad: "Audio processing has many techniques including FFT, spectrograms, and wavelets..."

            ### Contextual Precision (Needs Ground Truth)
            **Question:** Did we retrieve the right documents?

            **What it checks:**
            - Relevant documents ranked higher
            - Irrelevant documents ranked lower
            - Quality of retrieval system

            **Why it matters:** Good precision = less noise, more signal

            ### Contextual Recall (Needs Ground Truth)
            **Question:** Did we find all the important information?

            **What it checks:**
            - All necessary facts were retrieved
            - No critical information missing
            - Completeness of retrieval

            **Why it matters:** Good recall = comprehensive answers

            ---

            ## 🎯 Best Practices

            ### For Single Query Evaluation
            1. **Always provide ground truth** when evaluating - you get 4 metrics instead of 2
            2. **Be specific** in ground truth - "MFCC = Mel Frequency Cepstral Coefficients" is better than "It's an audio thing"
            3. **Check the report** - read why scores are high or low
            4. **Provide feedback** - use thumbs up/down to track satisfaction

            ### For Batch Evaluation
            1. **Collect 3-5 queries minimum** - more data = better insights
            2. **Use diverse questions** - test different aspects of your documents
            3. **Provide corrections** - builds your ground truth dataset automatically
            4. **Run regularly** - track improvement over time
            5. **Compare results** - see if changes improve performance

            ### For Ground Truth
            **Good Ground Truth:**
            - Accurate and complete
            - Based on your documents
            - Clear and concise
            - Consistent in quality

            **Bad Ground Truth:**
            - Too vague: "It's a thing"
            - Too detailed: Full paragraph when one sentence would do
            - Inconsistent: Sometimes detailed, sometimes brief
            - Inaccurate: Doesn't match documents

            ---

            ## 🔧 Troubleshooting

            ### "System Not Initialized"
            **Problem:** Can't ask questions
            **Solution:**
            1. Go to "Document Setup" tab
            2. Upload documents or browse for them
            3. Click "Initialize System"
            4. Wait for success message

            ### "Evaluation Failed" or No Scores
            **Problem:** Evaluation returns no scores
            **Solutions:**
            1. Check console for detailed errors
            2. Verify Cohere API key is valid
            3. Check internet connection
            4. Wait 5 minutes (rate limits)
            5. Try without ground truth first

            ### "No Ground Truth Data Available"
            **Problem:** Can't run batch evaluation
            **Solutions:**
            1. Submit queries with ground truth (check evaluation box)
            2. Or provide corrections to existing queries
            3. Need at least 1 query with ground truth
            4. Use helper: `prepare_for_batch_evaluation()`

            ### Evaluation Takes Too Long
            **Normal Timing:**
            - Single query without ground truth: 5-10 seconds
            - Single query with ground truth: 10-20 seconds
            - Batch (3 queries): 30-60 seconds
            - Batch (5 queries): 50-100 seconds

            **If slower:**
            - Check internet connection
            - Cohere API might be slow (wait and retry)
            - Rate limits (wait 5-10 minutes)

            ### Low Evaluation Scores
            **If Faithfulness is low:**
            - System may be hallucinating
            - Check retrieved documents quality
            - May need better document chunking

            **If Answer Relevancy is low:**
            - Answers may be off-topic
            - Try rephrasing questions
            - Check if documents contain relevant info

            **If Contextual Precision is low:**
            - Retrieval finding wrong documents
            - Try adjusting Top-K parameter
            - Enable reranking if disabled

            **If Contextual Recall is low:**
            - Missing important information
            - Increase Top-K to retrieve more
            - Check if info exists in documents

            ---

            ## 💾 Data Management

            ### Exporting Data
            Go to "Feedback Dashboard" → Click "Export All Data"

            **You get 3 files:**
            1. **feedback_data.json** - All user feedback (thumbs, ratings, corrections)
            2. **evaluation_results.json** - All DeepEval scores and results
            3. **ground_truth.json** - All questions with correct answers

            **Use cases:**
            - Analyze trends in Excel/Python
            - Track system improvement
            - Build ML datasets
            - Generate reports

            ### Clearing Data
            **History Tab** → "Clear History" - Removes all queries and feedback

            **Warning:** This cannot be undone! Export data first if needed.

            ---

            ## 🆘 Getting Help

            ### Check Console Output
            Most errors show detailed information in the console. Look for:
            - ❌ Error messages
            - 📋 Tracebacks
            - ⚠️ Warnings

            ### Common Error Messages

            **"API Key Error"**
            → Get new key: https://dashboard.cohere.com/api-keys

            **"Rate Limit Exceeded"**
            → Wait 5-10 minutes, check usage limits

            **"Context is empty"**
            → Documents not loaded properly, reinitialize

            **"Model generation failed"**
            → Check Cohere API status, verify internet

            ---

            ## 📞 Support & Resources

            **Created by:** Abdelaziz Ouayazza
            - 📧 Email: ouayazza.abdelaziz@gmail.com
            - 💼 LinkedIn: [Connect](https://www.linkedin.com/in/abdelaziz-ouayazza-61b19227b/)
            - 💻 GitHub: [abdelaziz2003vvb](https://github.com/abdelaziz2003vvb)

            **Useful Links:**
            - [DeepEval Docs](https://docs.confident-ai.com/)
            - [Cohere Documentation](https://docs.cohere.com/)
            - [LangChain Guide](https://python.langchain.com/)
            - [FAISS Documentation](https://faiss.ai/)

            ---

            **Version:** 2.0 with DeepEval
            **Last Updated:** December 2024
            **License:** Educational Use
            """)

# =============================================================================
# LAUNCH
# =============================================================================

print("\n" + "="*70)
print("🚀 LAUNCHING ENHANCED RAG SYSTEM")
print("="*70)
print("\n✨ Features:")
print("  ✅ Document upload & processing (PDF, DOCX, TXT, MD)")
print("  ✅ FAISS vector search with Cohere reranking")
print("  ✅ DeepEval evaluation (4 metrics)")
print("  ✅ Human feedback collection (thumbs, ratings, corrections)")
print("  ✅ Feedback analytics dashboard")
print("  ✅ Batch evaluation with comprehensive reports")
print("  ✅ Data export (JSON format)")
print("  ✅ Complete documentation & help")
print("\n📊 DeepEval Metrics:")
print("  • Faithfulness - factual consistency")
print("  • Answer Relevancy - addresses question")
print("  • Contextual Precision - retrieval quality (needs GT)")
print("  • Contextual Recall - retrieval completeness (needs GT)")
print("\n🎯 Quick Start:")
print("  1. Document Setup → Upload/Browse → Initialize")
print("  2. Ask & Evaluate → Enter question → Get Answer")
print("  3. Enable evaluation → Add ground truth → Submit")
print("  4. Batch Evaluation → Run evaluation → View report")
print("\n" + "="*70 + "\n")

# Launch interface
demo.launch(
    share=True,
    debug=False,
    show_error=True
)


🚀 LAUNCHING ENHANCED RAG SYSTEM

✨ Features:
  ✅ Document upload & processing (PDF, DOCX, TXT, MD)
  ✅ FAISS vector search with Cohere reranking
  ✅ DeepEval evaluation (4 metrics)
  ✅ Human feedback collection (thumbs, ratings, corrections)
  ✅ Feedback analytics dashboard
  ✅ Batch evaluation with comprehensive reports
  ✅ Data export (JSON format)
  ✅ Complete documentation & help

📊 DeepEval Metrics:
  • Faithfulness - factual consistency
  • Answer Relevancy - addresses question
  • Contextual Precision - retrieval quality (needs GT)
  • Contextual Recall - retrieval completeness (needs GT)

🎯 Quick Start:
  1. Document Setup → Upload/Browse → Initialize
  2. Ask & Evaluate → Enter question → Get Answer
  3. Enable evaluation → Add ground truth → Submit
  4. Batch Evaluation → Run evaluation → View report


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e2de1796c24c54b1c1.gradio.live

This share link expires 